In [1]:
import pandas as pd
import numpy as np
import requests
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from pandas_profiling import ProfileReport


In [3]:
df_1 = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/1gvZ03uAL6THwd04Y98GtIj6SeAHiKyQY5UisuuyFSUs/export?format=csv', low_memory=False
)
df_2 = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/1wduqo5WyYmCpaGnE81sLNGU0VSodIekMfpmEwU0fGqs/export?format=csv', low_memory=False
)

df_mergeado = df_2.merge(df_1, left_on = 'id', right_on = 'id')
df_mergeado.sort_values(by=['id'], inplace=True, ascending=True)
df_mergeado.head()

,barrio,dia,direccion_viento_tarde,direccion_viento_temprano,horas_de_sol,humedad_tarde,humedad_temprano,id,llovieron_hamburguesas_hoy,mm_evaporados_agua,...,presion_atmosferica_temprano,rafaga_viento_max_direccion,rafaga_viento_max_velocidad,temp_max,temp_min,temperatura_tarde,temperatura_temprano,velocidad_viendo_tarde,velocidad_viendo_temprano,llovieron_hamburguesas_al_dia_siguiente
43694,Agronomía,2008-12-01,Oestenoroeste,Oeste,NaN,22.0,71.0,0,no,NaN,...,1007.7,Oeste,44.0,22.9,13.4,21.8,16.9,24.0,20.0,no
1607,Agronomía,2008-12-02,Oestesuroeste,Nornoreste,NaN,25.0,44.0,1,no,NaN,...,1010.6,Oestenoroeste,44.0,25.1,7.4,24.3,17.2,22.0,4.0,no
100483,Agronomía,2008-12-03,Oestesuroeste,Oeste,NaN,30.0,38.0,2,no,NaN,...,1007.6,Oestesuroeste,46.0,25.7,12.9,23.2,21.0,26.0,19.0,no
29574,Agronomía,2008-12-04,Este,Sureste,NaN,16.0,45.0,3,no,NaN,...,1017.6,Noreste,24.0,28.0,9.2,26.5,18.1,9.0,11.0,no
42669,Agronomía,2008-12-05,Noroeste,Estenoreste,NaN,33.0,82.0,4,no,NaN,...,1010.8,Oeste,41.0,32.3,17.5,29.7,17.8,20.0,7.0,no
